In [60]:
import os
import glob
import h5py
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sio 
import cv2
import pickle
import time
import random
import sys
from PIL import Image
from collections import namedtuple as nt

In [61]:
os.environ['KERAS_BACKEND'] = 'tensorflow'
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

In [62]:
from keras.models import Sequential
from keras.optimizers import SGD
from keras.utils import np_utils
from keras.models import Model
from keras.layers import Input
from keras.utils import data_utils, plot_model
from keras import backend as K
from keras.models import model_from_json
from keras.callbacks import TensorBoard
K.clear_session()

In [63]:
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.gpu_options.visible_device_list = "0"
set_session(tf.Session(config=config))


In [64]:
TYNAMO_HOME_DIR = '/nfs/tynamo/home/data/vision7/gdhody/chrono/'
BLITZLE_HOME_DIR = '/nfs/blitzle/home/data/vision5/gdhody/chrono/'
HOME_DIR = TYNAMO_HOME_DIR

In [65]:
EPOCH = str(80)
BATCH_SIZE = 128
LR, LR_DECAY = 0.01, 0.0000625
LOSS = 'mean_squared_error'
MODEL_NAME = 'URLSS'
MODEL_PATH = os.path.join(HOME_DIR, 'models', MODEL_NAME, 'model.json')
MODEL_WEIGHTS = os.path.join(HOME_DIR, 'models', MODEL_NAME, 'weights', 'EPOCH_' + EPOCH + '.h5')
HDF5_PATH = os.path.join(HOME_DIR, 'patch.hdf5')
PICKLE_PATH = os.path.join(HOME_DIR, 'patch.pkl')

In [66]:
# %run model_and_callback.ipynb
# %run patch_DataLoad_Binary.ipynb
from compare_model_and_callback import lstm_model, EpochCallback
from patch_DataLoad_Binary import ChronoDataSet

In [67]:
model_input, model_output = lstm_model(
                                        input_shape=(4, 80, 80, 3),
                                        resnet=False,
                                        dense_feature_to_compare=0,
                                        drop_feature=0,
                                        compare_layer=True,
                                        compare_layer_dense=512,
                                        drop_compare_dense=0,
                                        lstm_cells=[(0, True, 'sum')],
                                        many_to_many=False,
                                        dense_layer_after_lstm=0,
                                        multi_output=False,
                                        number_of_classes=1,
                                     )
model = Model(inputs=[model_input], outputs=[model_output])
output_layer_name = model.layers[-1].name
print model.summary()

Model Creation Started
Model Caffe Base Created
Model Compare Unit Finished
Model LSTM Unit Finished
Model Output Layers Finished
Model Creation Finished
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              (None, 4, 80, 80, 3) 0                                            
__________________________________________________________________________________________________
TD_conv1_96_7_3 (TimeDistribute (None, 4, 24, 24, 96 34944       input[0][0]                      
__________________________________________________________________________________________________
TD_mpool1_3_2 (TimeDistributed) (None, 4, 11, 11, 96 0           TD_conv1_96_7_3[0][0]            
__________________________________________________________________________________________________
TD_lrnorm1 (TimeDistributed)    (None, 4, 11, 11, 96 0

In [68]:
test_generator = ChronoDataSet(
                                        PICKLE_PATH,
                                        HDF5_PATH,
                                        batch_size=BATCH_SIZE,
                                        start_epoch=0,
                                        mode='TEST',
                                        dataset='UCF'
                                )
print "Test Data Generator Ready"

Test Data Generator Ready


In [69]:
sgd = SGD(lr=LR, decay=LR_DECAY, momentum=0.9, nesterov=True)
model.compile(
                loss={output_layer_name : LOSS},
                optimizer=sgd,
                metrics=['accuracy']
             )

In [70]:
model.load_weights(MODEL_WEIGHTS, by_name=True)
print "Weights from", str(EPOCH), "epoch Loaded", MODEL_WEIGHTS

Weights from 78 epoch Loaded /nfs/tynamo/home/data/vision7/gdhody/chrono/models/BINARY_CD512_LSTM128BI_1_LR05_LRD00005/weights/EPOCH_78.h5


In [ ]:
print "TESTING"
print model.metrics_names
print model.evaluate_generator(
                            generator=test_generator,
                            steps=test_generator.getBatchRunsPerEpoch(),
                            max_queue_size=9,
                            workers=3,
                            use_multiprocessing=True
)

['loss', 'acc']